In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 22 13:57:10 2023

@author: BigD
"""

#Import all moduels as required
import astropy as ap
from astropy.stats import poisson_conf_interval
from astropy.table import Table
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table
from statistics import mean
from statistics import stdev
from scipy.stats import (moyal, invgamma, lognorm, burr, beta, gamma, expon, exponpow, chi2, cauchy) 
from scipy import stats
from fitter import Fitter, get_common_distributions, get_distributions


# STEP 1 ORGANISE THE DATA

# Import the crossmatched data 
data = Table.read('/Users/BigD/cloudstor/Data/EMU - GAMA - G23')
data = data.to_pandas()

#G23_data = Table.read('C:/Users/BigD/cloudstor/Data/GAMA/GAMA - Michael Brown/GAMA_G23_v01_mu3_20191202.fits')
#G23_data = G23_data.to_pandas()

#import the EMU data
EMU_data = Table.read('/Users/BigD/cloudstor/Data/EMU/G23-ASKAP-EMUES-master-cat.fits', format='fits')
EMU_data= EMU_data.to_pandas()

#merge these two groups so that you have a ~40,000 sources table with 93 columns.
data = pd.merge(data, EMU_data, how="outer", on = 'Source_Name')
data = data[['Source_Name','RA_y','E_RA_y','DEC_y','E_DEC_y' ,'Total_flux_y','E_Total_flux_y','Z','NQ','Separation_x']]
data = data.rename(columns = {"RA_y":"RA", 'E_RA_y':'E_RA', 'DEC_y':'DEC', 'E_DEC_y':'E_DEC','Total_flux_y':'Total_flux','E_Total_flux_y':'E_Total_flux','Separation_x':'Separation' })

#del(EMU_data, data)

#We now have the crossmatched data set of EMU and G23, outlining:
print(list(data))

#create a column containing the rel error
data['rel_err']=data['E_Total_flux']/data['Total_flux']

#%% STEP 2 Visualise the data

from observationsofdataset import Visualiseradiodata

Visualiseradiodata(data)

#we can see that there needs to be a lot of completness corrections
    
#%% STEP 3
# Import required modules
import matplotlib.pyplot as plt
from completenesscorrection import getpdfs

# STEP 3: CREATE THE PDFS FOR MISSING REDSHIFT AND REDSHIFT ERRORS
# NOTE: THIS STEP WILL CREATE A DATAFRAME DF THAT IS POSITIVE FLUX AND Z>0

# 3.1: Create pdfs for the redshift
tol = 10  # The minimum number of data points in each bin to form a probability distribution function
n = 10  # The number of magnitude bins the data will be separated into
df, pdfs, labels, use_labels, bins = getpdfs(data, n, tol)

# 3.2: Show what portion of the data set was used to create pdf's
fig1, ax = plt.subplots(1, 1)
for label in labels:
    ax.hist(df[df['bin'] == label]['mag'], alpha = 1, rwidth = 1, 
            range = (df['mag'].min(), df['mag'].max()), bins = 200, label = label)
ax.set_xlabel('$Mag_{AB}$')
ax.set_ylabel('Counts')
plt.suptitle('The different magnitudes that created the pdfs')
plt.legend(fontsize = 10)

datapoint_count = sum(df['bin'].str.contains(label).sum() for label in use_labels)
percentage = "{0:.3%}".format(datapoint_count / len(df))
print(f"pdf's were calculated from {percentage} of the datapoints that have a positive flux and greater than zero redshift")


#%% STEP 4 FILL IN THE MISSING DATA USING THE CREATED PDF'S
# Import required modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fillinmissing import Fillinmissing

# STEP 4: FILL IN THE MISSING DATA USING THE CREATED PDF'S

# Calculate the magnitudes for each data point
# Convert everything to AB magnitude
data['mag'] = 25 - 2.5 * np.log10(data['Total_flux'])

# Use the bins that the pdf's were created on, however add an upper and lower bound [ 0,50]
# so to capture anything that falls outside the range covered in the spectro sample
bins = [0] + bins + [50]  # Add lower and upper boundaries
# Also create an upper bound for our labels
labels.append(str(labels[-1].replace('<', ">")))

# Bin in accordance with the labels for which the pdf's were calculated
data['bin'] = pd.cut(data['mag'], bins=bins, labels=labels)
# Check the distribution of the bins
print(data['bin'].value_counts(dropna=False))

# Count what percentage of data the pdf's don't account for
datapoint_count = sum(data['bin'].value_counts(dropna=False)[label] for label in use_labels)
percentage = "{0:.3%}".format(datapoint_count / len(data))
print(f"The pdf's can be utilized for {percentage} of the EMU data")

# 4.1: Fill in the missing data and draw comparison of the mock and given data
df = Fillinmissing(data, bins, labels, use_labels, pdfs)

# Note: Losing about 590 readings with only a 98.475% coverage
# For now, this is good enough. Let's continue with the 98.475% data for the luminosity functions.
df = df[df['Z'] > 0]

# Display a magnitude vs redshift graph for the filled-in data
fig2, ax = plt.subplots(1, 1)
ax.scatter(df['Z'], np.log10(df['Total_flux']))
ax.set_xlabel('Redshift')
ax.set_ylabel('Flux (Jy)')


#%%
# Import required modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.cosmology import WMAP9 as cosmo

# STEP 5: Calculate the luminosity's for everything

# 5.1: Find the turnaround of the flux
fig1, ax1 = plt.subplots()
counts, bin_edges = np.histogram(np.log10(df['Total_flux']), bins=200)
ax1.hist(np.log10(df['Total_flux']), bins=200)
ax1.set_title('The total flux as measured by EMU')
ax1.set_xlabel('$Log_{10} S (Jansky)$')
ax1.set_ylabel('counts') 

flux_limit_idx = np.argmax(counts)
flux_limit = 10**bin_edges[flux_limit_idx]
print(f'The flux limit is {flux_limit:.5f}')

# 5.2: Reduce the data set to those values with only a positive redshift
df = df[df['Z'] > 0].reset_index(drop=True)

fig2, ax1 = plt.subplots()
ax1.scatter(df['Z'], np.log10(df['Total_flux']))
ax1.set_title('The distribution of flux readings across the EMU GAMA Survey')
ax1.set_xlabel('Redshift')
ax1.set_ylabel('$Log_{10} S (Janksy)$')

# 5.4: Calculate the luminosity for everything
flux_limit = 200e-6  # According to Gürkan the flux limits is 0.2mJy or 200uJy
radio_index = -0.7
EMU_Freq = 0.8875  # Ghz (997 Mhz)
Comp_Freq = 1.4  # Ghz

# Calculate luminosity distance and add it to dataframe
df['ld'] = [cosmo.luminosity_distance(z).value * 3.086e22 for z in df['Z']]

# Calculate the luminosity at 1.4Ghz
lum_factor = 4*np.pi*(df['ld']**2)/((1+df['Z'])**(1+radio_index)*(EMU_Freq/Comp_Freq)**radio_index)
df['lum'] = lum_factor * (df['Total_flux']*1e-26)

Min_lum_line = lum_factor * (flux_limit*1e-26)

# 5.4: Plot the results
fig3, [ax1, ax2] = plt.subplots(1, 2)
fig3.suptitle('The distribution of readings across the EMU GAMA Survey', fontsize=15)

ax1.scatter(df['Z'], np.log10(df['lum']))
ax1.scatter(df['Z'], np.log10(Min_lum_line), s=2, label=f'{flux_limit*1000}mJy')
ax1.set_xlabel('Redshift')
ax1.set_ylabel('$Log_{10} L (Watts)$')
ax1.legend(fontsize=10)

ax2.scatter(df['Z'], np.log10(df['Total_flux']), c='r')
ax2.set_xlabel('Redshift')
ax2.set_ylabel('$Log_{10} S (Jansky)$')

#%% STEP 6 Calculate Vmax using Novak Vmax method
# Import required modules
import numpy as np
import pandas as pd
from astropy.cosmology import WMAP9 as cosmo

# Define redshift bins and labels
redshift_bins = [0.0, 0.1, 0.4, 0.6, 0.8, 1.0, 1.3, 1.6, 2.0]
z_labels = ['<' + str(bin_edge) for bin_edge in redshift_bins]

# Assign each data point to a redshift bin
df['z_bin'] = pd.cut(df['Z'], bins=redshift_bins, labels=z_labels[1:])
df['min_z'] = pd.cut(df['Z'], bins=redshift_bins, labels=redshift_bins[:-1])

# Print the distribution of the bins
print(df['z_bin'].value_counts(dropna=False))

# Choose Vmax method
Vmax_method = "Observable"

if Vmax_method == "Novak":
    survey_area = 82.7 / 41253  # Portion of the sky surveyed (82.7 deg^2).
    z = df['Z']
    bin_num = [next((i for i, bin_edge in enumerate(redshift_bins) if value < bin_edge), -1) for value in z]
    
    # Calculate the volume for each redshift bin
    Vbin = [(4*np.pi/3) * (cosmo.comoving_distance(z2).value**3 - cosmo.comoving_distance(z1).value**3)
            for z1, z2 in zip(redshift_bins[:-1], redshift_bins[1:])]
    Vbin.append(0)  # Add zero volume for things outside the redshift bins
    
    # Calculate Vmax for each galaxy
    df['Vmax'] = [Vbin[i] * survey_area for i in bin_num]

elif Vmax_method == "Observable":
    dmax = np.sqrt(df['lum'] * ((1 + df['Z']) ** (1 + radio_index)) / ((flux_limit * 1e-26) * (4 * np.pi)))
    dmax *= 3.24078e-23  # Convert to Mpc
    df['dmax'] = dmax
    min_d = [cosmo.comoving_distance(bin_edge).value for bin_edge in redshift_bins[:-1]]
    df['dmin'] = [cosmo.comoving_distance(min_z).value for min_z in df['min_z']]
    df['dactual'] = [cosmo.comoving_distance(z).value for z in df['Z']]
    
    survey_area = 82.7 / 41253  # Portion of the sky surveyed (82.7 deg^2).
    df['Vmax'] = (4 / 3) * np.pi * (df['dmax'] ** 3 - df['dmin'] ** 3) * survey_area
    
    
    
#%%
# STEP 7: Classify the data

# 7.1 Create our labels to slice up the data
# Generate labels of the form <0.4, <0.6 etc, that will bin each data point starting at 0.0
z_labels = ['<' + str(bin_edge) for bin_edge in redshift_bins]

# Assign each data point to a redshift bin
df['z_bin'] = pd.cut(df['Z'], bins=redshift_bins, labels=z_labels[1:])
df['min_z'] = pd.cut(df['Z'], bins=redshift_bins, labels=redshift_bins[:-1])

# Check the distribution of the bins
print(df['z_bin'].value_counts(dropna=False))


#%%
#Step 8 Calculate PHI
import pandas as pd
import astropy as ap
from astropy.stats import poisson_conf_interval

# Define your settings
n_bins = 10
lit = "Novak"

# Define redshift bins based on literature
if lit == "Enia":
    redshift_bins = [0.0, 0.4, 0.7, 1.0, 2.0, 3.0, 10.0]
elif lit == "Novak":
    redshift_bins = [0.0, 0.1, 0.4, 0.6, 0.8, 1.0, 1.3, 1.6, 2.0]

# Create labels for the bins
z_labels = list(zip(redshift_bins[:-1], redshift_bins[1:]))
z_labels = ['<' + str(bin_edge) for bin_edge in redshift_bins]

# Assign each data point to a bin
df['z_bin'] = pd.cut(df['Z'], bins=redshift_bins, labels=z_labels[1:])
df['min_z'] = pd.cut(df['Z'], bins=redshift_bins, labels=redshift_bins[:-1])

# Initialize arrays to store results
Phi_arr, Lum_arr, error_arr, counts_arr, upper_err_arr, lower_err_arr = ([] for _ in range(6))

# Loop over each redshift bin
for label in z_labels:
    zslice = df[df['z_bin'] == label][['lum', 'Total_flux', 'Vmax', 'Z']]
    
    # Identify and remove outliers
    mean, std = np.mean(zslice['lum']), np.std(zslice['lum'])
    outliers = zslice[(zslice['lum'] > mean + 3*std) | (zslice['lum'] < mean - 3*std)]
    zslice = zslice.drop(outliers.index)
    print(f"For redshift between {label[0]} and {label[1]}, {len(outliers)} outliers were removed from a list of {len(zslice)+len(outliers)}.")

    # Bin luminosities
    counts, bin_edges = np.histogram(np.log10(zslice['lum']), bins=n_bins)
    print(f"The number of galaxies in each bin is {counts}. \n")

    # Sort zslice by luminosity
    zslice = zslice.sort_values(by='lum')

    # Initialize arrays for this bin
    phi, err, l_half, l_average = ([] for _ in range(4))
    
    # Loop over each luminosity bin
    for n, count in enumerate(counts):
        if count > 0:
            # Perform calculations
            delta_l = bin_edges[n+1] - bin_edges[n]
            l_half.append((bin_edges[n+1] + bin_edges[n]) / 2)
            l_average.append(np.log10(np.mean(zslice['lum'][:count])))
            phi_val = 1 / delta_l * sum(1 / zslice['Vmax'][:count])
            phi.append(phi_val)
            error = 1 / delta_l * np.sqrt(sum(1 / zslice['Vmax'][:count]**2))
            err.append(error if count > 10 else phi_val * (1 - ap.stats.poisson_conf_interval(count)[0] / count))
            zslice = zslice[count:]

    # Store results
    counts_arr.append(counts)
    Phi_arr.append(phi)
    error_arr.append(err)
    Lum_arr.append(l_average)
    upper_err_arr.append(np.add(phi, err))
    lower_err_arr.append(np.maximum(0, np.subtract(phi, err)))

print("Analysis complete.")

#%%

import matplotlib.pyplot as plt
import numpy as np

#4.3 Graph the values of Phi and Lum for all redshift bins
counter = len(z_labels)-1 #Find how many redshift bins we wish to graph

#4.3.1 Plot
fig, ax = plt.subplots(2,int(counter/2), sharex=True, sharey=True)
fig.suptitle('Luminosity functions for various redshift \n slices using the EMU-G23 data', fontsize=10)
fig.text(0.5, 0.02, r'$logL_{1.4GHz}[W Hz^{-1}]$', ha='center', fontsize=10)
fig.text(0.02, 0.5, r'$log\Phi[Mpc^{-3}dex^{-1}]$', va='center', rotation='vertical', fontsize=10)
plt.subplots_adjust(bottom=0.15, left=0.11, top=0.9, wspace=0, hspace=0)
plt.yscale('log')

for ii in range(counter): #RUN THE LOOP THROUGH THE LABELLED/FILTERED INSTEAD OF SLICES
    textstr = str(z_labels[ii][1:]) + '<z' + str(z_labels[ii+1])
    print(ii)

    # Create out asymetricerror
    asymetricerror = np.array([lower_err_arr[ii], upper_err_arr[ii]])

    # Determine row and column index for plotting
    row = ii // int(counter/2)
    col = ii % int(counter/2)
    
    # Plot the resulting phi and Luminosity on the two rows of the figure
    ax[row, col].scatter(Lum_arr[ii], Phi_arr[ii], label='EMU G23')
    ax[row, col].tick_params(axis='both', labelsize=10)
    ax[row, col].label_outer()
    ax[row, col].grid()
    ax[row, col].text(0.02, 0.1, textstr, transform=ax[row, col].transAxes, fontsize=10,
                      verticalalignment='bottom', color='red',
                      bbox=dict(facecolor='white', edgecolor='red', pad=2.0))
    
    if error_bars == "y":
        ax[row, col].errorbar(Lum_arr[ii], Phi_arr[ii], yerr=asymetricerror, ls='none')

    print('The Phi values for each bins are ', ['%.2f' % np.log10(elem) for elem in Phi_arr[ii]])
    print("The number of points in each bin are ", counts_arr[ii])
    print('The data is plotted at luminosities of ', ['%.2f' % elem for elem in Lum_arr[ii]])
    print('\n')

# Add additional overlays here, such as Novak and Enia findings

handles, labels = ax[0, 1].get_legend_handles_labels()
fig.legend(handles, labels, bbox_to_anchor=(1, 1.03), loc='upper right', fontsize=9)
    
    
#Novak Findings Overlay
if lit=="Novak":
        Novak_L = [[21.77,22.15,22.46,22.77,23.09,23.34],[22.29,22.54,22.80,23.04,23.31,23.68],
                   [22.61,22.84,23.11,23.40,23.71,24.06],[22.85,23.05,23.30,23.54,23.81,24.11],
                   [23.10,23.31,23.57,23.84,24.06,24.38],[23.32,23.53,23.81,24.15,24.39,24.82],
                   [23.55,23.72,23.98,24.28,24.536,24.90]] #Need to change these to Log Luminosities
       
        Novak_Phi = [[-2.85,-2.88,-3.12,-3.55,-4.05,-4.63],[-2.97,-3.19,-3.33,-3.67,-4.32,-5.05],
                     [-2.89,-3.13,-3.47,-3.99,-4.68,-5.43],[-3.01,-3.13,-3.45,-3.85,-4.31,-4.89],
                     [-3.19,-3.42,-3.86,-4.15,-4.74,-5.25],[-3.36,-3.55,-4.10,-4.53,-5.30,-5.94],
                     [-3.47,-3.66,-4.15,-4.56,-5.06,-5.86]] 
        Novak_Phi = [np.power(10, x) for x in Novak_Phi]
        
        for i, (L, Phi) in enumerate(zip(Novak_L, Novak_Phi)):
            row = 0 if i < 4 else 1
            col = i if row == 0 else i - 4
            ax[row, col].scatter(L, Phi, label = "Novak 2017" if i==0 else None)
        
#Enia Findings Overlay
if lit =="Enia":  
    Enia_L=[[21.48,21.99,22.42,22.84],[22.16,22.55,22.96,23.38],
            [22.58,22.88,23.26,23.63],[23.04,23.57,23.96,24.35],
            [23.59,23.94,24.27,24.60],[23.70,24.09,24.27,24.69]]
    Enia_Phi=[[-2.40,-2.77,-3.20,-3.42],[-2.38,-2.75,-3.19,-3.80],
              [-2.54,-2.84,-3.19,-3.99],[-2.48,-3.55,-4.16,-4.72],
              [-2.94,-3.72,-4.26,-4.81],[-3.26,-3.98,-4.52,-4.98]]
                    
                        
    for i, (L, Phi) in enumerate(zip(Enia_L, Enia_Phi)):
        row = 0 if i < 3 else 1
        col = i if row == 0 else i - 3
        ax[row, col].scatter(L, Phi, label = "Enia 2022" if i==0 else None)
    
# Add legend
handles, labels = ax[0,0].get_legend_handles_labels()
fig.legend(handles, labels, bbox_to_anchor = (1,1.03), loc='upper right', fontsize=9)
plt.show()